<a href="https://colab.research.google.com/github/IraJasper/DuLabCode/blob/main/NV_measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import TimeTagger
import numpy as np
import time
from scipy.signal import butter, cheby1, filtfilt
import matplotlib.pyplot as plt
from hardware.timetagger_counter import HWRecorderMode
import itertools
from scipy.optimize import curve_fit
from datetime import date
from tqdm import tqdm
import os

ModuleNotFoundError: No module named 'TimeTagger'

In [11]:
def get_user_input_for_dict(data_dict):
    """
    Prompts the user to input values for each key in a given dictionary.

    Args:
        data_dict (dict): The dictionary whose values need to be updated by user input.

    Returns:
        dict: The dictionary with updated values based on user input.
    """
    updated_dict = data_dict.copy() # Work on a copy to avoid modifying the original dict directly

    for key, value in updated_dict.items():
        user_input = input(f"Enter value for '{key}' (current value: {value}): ")
        updated_dict[key] = user_input
    return updated_dict

In [ ]:
def get_data(analyze_window_start,analyze_window_length):

    counts_raw = counter.getData()
    counts = np.sum(counts_raw[:,analyze_window_start:analyze_window_start+analyze_window_length],1)

    return counts

In [ ]:
class fitting_methods:
    @staticmethod
    def pulsed_ODMR(x, y, initial_guess=None):
        """
        Fits Pulsed ODMR data.
        Returns: resonance frequency, linewidth, contrast
        """
        # Insert fitting logic here (e.g., Lorentzian fit)
        print("Fitting Pulsed ODMR data...")
        fitted_freq = 2.87e9 # Placeholder
        return fitted_freq

    @staticmethod
    def CW_ODMR(x, y, initial_guess=None):
        """
        Fits CW ODMR data.
        Returns: resonance frequency, linewidth, contrast
        """
        # Insert fitting logic here
        print("Fitting CW ODMR data...")
        fitted_freq = 2.87e9 # Placeholder
        return fitted_freq

    @staticmethod
    def rabi(x, y, initial_guess=None):
        """
        Fits Rabi oscillation data.
        Returns: pi_pulse duration, decay constant
        """
        # Insert fitting logic here (e.g., damped sine wave)
        print("Fitting Rabi data...")
        pi_pulse = 50 # Placeholder ns
        return pi_pulse

    @staticmethod
    def rabi_IQ(x, y, initial_guess=None):
        """
        Fits Rabi IQ data.
        """
        # Insert fitting logic here
        print("Fitting Rabi IQ data...")
        return None

    @staticmethod
    def T1(x, y, initial_guess=None):
        """
        Fits T1 relaxation data.
        Returns: T1 time
        """
        # Insert fitting logic here (e.g., exponential decay)
        print("Fitting T1 data...")
        T1_time = 1e6 # Placeholder ns
        return T1_time

    @staticmethod
    def T1_SCC(x, y, initial_guess=None):
        """
        Fits T1 SCC data.
        """
        # Insert fitting logic here
        print("Fitting T1 SCC data...")
        return None

    @staticmethod
    def T2(x, y, initial_guess=None):
        """
        Fits T2 (Hahn Echo or XY8) data.
        Returns: T2 time
        """
        # Insert fitting logic here (e.g., exponential decay)
        print("Fitting T2 data...")
        T2_time = 100e3 # Placeholder ns
        return T2_time

In [ ]:
class MeasurementDefinitions:
    def pulsed_ODMR(self, parameters):
        pi_pulse = parameters['pi_pulse']
        freq_dev = parameters['freq_dev']
        num_freq = parameters['num_freq']

        laser_length = 1500 #ns
        wait_length = 400 #ns
        analog_delay = 000 #ns
        pulsed_length = pi_pulse+laser_length+wait_length #ns

        mw_mod_length = pulsed_length*1 #ns
        meas_time  = 0

        binwidth = 1 # ns
        binwidth_ps = binwidth*1000 #ps
        n_bins = laser_length/binwidth

        mw_mod_output = np.linspace(-1*freq_dev/4e7,1*freq_dev/4e7,num_freq)

        block_mw_mod = []
        block_mw_switch = []
        block_laser = []
        block_gate = []

        block_mw_switch.extend([(analog_delay,0)])
        block_laser.extend([(analog_delay,0)])
        block_gate.extend([(analog_delay,0)])
        for i in range(num_freq):
            block_mw_mod.extend([(mw_mod_length,mw_mod_output[i])])
            block_mw_switch.extend([(pi_pulse, 1), (laser_length, 0), (wait_length, 0)]) #2-mw switch
            block_laser.extend([(pi_pulse, 0), (laser_length, 1), (wait_length, 0)]) #0-laser
            block_gate.extend([(pi_pulse, 0),(laser_length, 1), (wait_length, 0)]) #5-TT start
            meas_time+= pi_pulse+laser_length+wait_length

        block_mw_mod.extend([(analog_delay,0)])
        meas_time+= analog_delay

        sequences = {}
        # Use names instead of (type, channel)
        sequences['mw_mod'] = block_mw_mod
        sequences['mw_switch'] = block_mw_switch
        sequences['laser'] = block_laser
        sequences['counter_gate'] = block_gate
        sequences['sync'] = [(meas_time,0),(40,1)]

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': int(n_bins),
            'n_histograms': int(num_freq)
        }

        return sequences, tagger_config, meas_time

    def CW_ODMR(self, parameters):
        pi_pulse = parameters['pi_pulse']
        freq_dev = parameters['freq_dev']
        num_freq = parameters['num_freq']

        laser_length = 1500 #ns
        wait_length = 400 #ns
        analog_delay = 000 #ns
        pulsed_length =laser_length+wait_length #ns,CW

        mw_mod_length = pulsed_length*1 #ns
        meas_time  = 0

        binwidth = 1 # ns
        binwidth_ps = binwidth*1000 #ps
        n_bins = laser_length/binwidth

        mw_mod_output = np.linspace(-1*freq_dev/4e7,1*freq_dev/4e7,num_freq)

        block_mw_mod = []
        block_mw_switch = []
        block_laser = []
        block_gate = []

        block_mw_switch.extend([(analog_delay,0)])
        block_laser.extend([(analog_delay,0)])
        block_gate.extend([(analog_delay,0)])
        for i in range(num_freq):
            block_mw_mod.extend([(mw_mod_length,mw_mod_output[i])])
            block_mw_switch.extend([ (laser_length, 1), (wait_length, 0)]) #2-mw switch
            block_laser.extend([(laser_length, 1), (wait_length, 0)]) #0-laser
            block_gate.extend([(laser_length, 1), (wait_length, 0)]) #5-TT start
            meas_time+= laser_length+wait_length

        block_mw_mod.extend([(analog_delay,0)])
        meas_time+= analog_delay

        sequences = {}
        sequences['mw_mod'] = block_mw_mod
        sequences['mw_switch'] = block_mw_switch
        sequences['laser'] = block_laser
        sequences['counter_gate'] = block_gate
        sequences['sync'] = [(meas_time,0),(40,1)]

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': int(n_bins),
            'n_histograms': int(num_freq)
        }

        return sequences, tagger_config, meas_time

    def rabi(self,parameters):
        laser_time = parameters['laser_time']
        wait_time = parameters['wait_time']
        tau_start = parameters['tau_start']
        tau_end = parameters['tau_end']
        num_points = parameters['num_points']

        taus = np.linspace(tau_start,tau_end,num_points)

        block_laser = []
        block_counter = []
        block_mw = []

        meas_time = 0

        for tau in taus:
            block_laser.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_counter.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_mw.extend([(tau,1),(laser_time,0),(wait_time,0)])
            meas_time += laser_time+wait_time+tau

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter_gate'] = block_counter
        sequences['mw'] = block_mw
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points
        }

        return sequences, tagger_config, meas_time

    def rabi_IQ(self,parameters):
        laser_time = parameters['laser_time']
        wait_time = parameters['wait_time']
        tau_start = parameters['tau_start']
        tau_end = parameters['tau_end']
        num_points = parameters['num_points']
        phi = parameters['phi']

        taus = np.linspace(tau_start,tau_end,num_points)

        block_laser = []
        block_counter = []
        block_I = []
        block_Q = []

        meas_time = 0

        for tau in taus:
            block_laser.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_counter.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_I.extend([(tau,0.5*np.sin(phi)),(laser_time,0),(wait_time,0)])
            block_Q.extend([(tau,0.5*np.cos(phi)),(laser_time,0),(wait_time,0)])
            meas_time += laser_time+wait_time+tau

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter_gate'] = block_counter
        sequences['I'] = block_I
        sequences['Q'] = block_Q
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points
        }

        return sequences, tagger_config, meas_time

    def T1(self,parameters):
        num_points = parameters['num_points']
        start_time = parameters['start_time'] #ns
        stop_time = parameters['stop_time'] #ns
        delay_time = np.geomspace(start_time, stop_time, num_points) #ns
        pi_pulse = parameters['pi_pulse'] #ns
        laser_time = parameters['laser_time'] # 1.6e3 #ns
        wait_time = parameters['wait_time'] # 500 #ns

        meas_time = 0
        block_laser = []

        block_mw = []

        for i in range(num_points):
            time_trace = [delay_time[i],pi_pulse,laser_time,wait_time,
                          delay_time[i],pi_pulse,laser_time,wait_time]
            laser_on_off = [0,0,1,0,0,0,1,0]
            mw_on_off = [0,0,0,0,0,1,0,0,]
            block_laser.extend(list(zip(time_trace,laser_on_off)))
            block_mw.extend(list(zip(time_trace,mw_on_off)))
            meas_time += sum(time_trace)

        block_counter = block_laser
        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40
        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter_gate'] = block_counter
        sequences['mw'] = block_mw
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points*2
        }

        return sequences, tagger_config, meas_time

    def T1_SCC(self,parameters):
        num_points = parameters['num_points']
        start_time = parameters['start_time'] #ns
        stop_time = parameters['stop_time'] #ns
        delay_time = np.geomspace(start_time, stop_time, num_points) #ns

        pi_pulse = parameters['pi_pulse'] #ns

        # config
        G_inti_time = parameters['G_inti_time']
        G_delay = parameters['G_delay']
        Y_read_time = parameters['Y_read_time']
        Y_delay = parameters['Y_delay']
        R_ionize_time = parameters['R_ionize_time']
        R_delay = parameters['R_delay']

        meas_time = 0

        block_G = []
        block_Y = []
        block_R = []
        block_mw = []
        block_counter = []
        for i in range(num_points):
            time_trace = [G_inti_time,G_delay,delay_time[i],pi_pulse,
                          R_ionize_time,R_delay,Y_read_time,Y_delay,
                          G_inti_time,G_delay,delay_time[i],pi_pulse,
                          R_ionize_time,R_delay,Y_read_time,Y_delay]
            G_on_off = [1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
            Y_on_off = [0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0]
            R_on_off = [0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0]
            mw_on_off = [0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0]
            counter_on_off = [0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1] # Due to the delay of yellow laser, the counter should be on during Y_delay to read the PL
            block_counter.extend(list(zip(time_trace,counter_on_off)))
            block_G.extend(list(zip(time_trace,G_on_off)))
            block_Y.extend(list(zip(time_trace,Y_on_off)))
            block_R.extend(list(zip(time_trace,R_on_off)))
            block_mw.extend(list(zip(time_trace,mw_on_off)))
            meas_time += sum(time_trace)

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['G'] = block_G
        sequences['Y'] = block_Y
        sequences['R'] = block_R
        sequences['mw'] = block_mw
        sequences['sync'] = block_sync
        sequences['counter_gate'] = block_counter

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e5,
            'n_bins': (Y_read_time+Y_delay)/1e2,
            'n_histograms': num_points*2
        }

        return sequences, tagger_config, meas_time

    def T2(self,parameters):
        xy_num = parameters['xy_num']
        xy_repetition = parameters['xy_repetition']
        start_phase = parameters['start_phase']
        laser_time = parameters['laser_time']
        wait_time = parameters['wait_time']
        mw_wait = parameters['mw_wait']
        pi_pulse = parameters['pi_pulse']
        num_points = parameters['num_points']
        tau_start = parameters['tau_start']
        tau_end = parameters['tau_end']
        taus = np.linspace(tau_start,tau_end,num_points)

        def xy_analog_on_off_one_block(i):
            if i % 2 == 0:
                A0_to_extend = [0,0.5,0]
                A1_to_extend = [0,0,0]
            elif i % 2 == 1:
                A0_to_extend = [0,0,0]
                A1_to_extend = [0,0.5,0]
            return A0_to_extend, A1_to_extend

        def xy_analog_on_off_four_block():
            A0_on_off_four = []
            A1_on_off_four = []
            for i in range(4):
                A0_to_extend,A1_to_extend = xy_analog_on_off_one_block(i)
                A0_on_off_four.extend(A0_to_extend)
                A1_on_off_four.extend(A1_to_extend)
            return A0_on_off_four,A1_on_off_four

        def xy_analog_on_off():
            A0_on_off_all = []
            A1_on_off_all = []

            if start_phase == 'X' or start_phase == 'x':
                phase = [0.5,0]
            elif start_phase == 'Y' or start_phase == 'y':
                phase = [0,0.5]
            elif start_phase == '-X' or start_phase == '-x':
                phase = [-0.5,0]
            elif start_phase == '-Y' or start_phase == '-y':
                phase = [0,-0.5]

            A0_on_off = [phase[0],0]
            A1_on_off = [phase[1],0]
            A0_on_off_four,A1_on_off_four = xy_analog_on_off_four_block()

            for _ in range(xy_repetition):
                if xy_num == 1:
                    A0_to_extend,A1_to_extend = xy_analog_on_off_one_block(0)
                    A0_on_off.extend(A0_to_extend)
                    A1_on_off.extend(A1_to_extend)
                elif xy_num == 4:
                    A0_on_off.extend(A0_on_off_four)
                    A1_on_off.extend(A1_on_off_four)
                elif xy_num == 8:
                    A0_on_off.extend(A0_on_off_four+A1_on_off_four)
                    A1_on_off.extend(A1_on_off_four+A0_on_off_four)
                elif xy_num == 16:
                    A0_on_off_four_conjugate = [-element for element in A0_on_off_four]
                    A1_on_off_four_conjugate = [-element for element in A1_on_off_four]
                    A0_on_off.extend(A0_on_off_four+A1_on_off_four+A0_on_off_four_conjugate+A1_on_off_four_conjugate)
                    A1_on_off.extend(A1_on_off_four+A0_on_off_four+A1_on_off_four_conjugate+A0_on_off_four_conjugate)

            if end_phase == 'X' or end_phase == 'x':
                phase = [0.5,0]
            elif end_phase == 'Y' or end_phase == 'y':
                phase = [0,0.5]
            elif end_phase == '-X' or end_phase == '-x':
                phase = [-0.5,0]
            elif end_phase == '-Y' or end_phase == '-y':
                phase = [0,-0.5]

            A0_on_off.extend([phase[0],0,0,0])
            A1_on_off.extend([phase[1],0,0,0])


            return A0_on_off,A1_on_off

        def get_time_trace(tau):
            time_trace = [pi_pulse/2,mw_wait]
            for _ in range(xy_repetition):
                for _ in range(xy_num):
                    time_trace.extend([tau/(2*xy_num*xy_repetition),pi_pulse,tau/(2*xy_num*xy_repetition)])
            time_trace.extend([pi_pulse/2,mw_wait,laser_time,wait_time])
            return time_trace

        def xy_block(tau):
            time_trace = get_time_trace(tau)
            I_block = []
            Q_block = []

            I_on_off,Q_on_off = xy_analog_on_off()

            for i in range(len(time_trace)):
                I_block.extend([(time_trace[i],I_on_off[i])])
                Q_block.extend([(time_trace[i],Q_on_off[i])])

            laser_block = [(time,0) for time in time_trace]
            laser_block[-2] = (laser_time,1)
            counter_block = laser_block

            return I_block,Q_block,laser_block,counter_block


        block_laser = []
        block_counter = []
        block_I = []
        block_Q = []
        meas_time = 0
        for tau in taus:
            for end_phase in ['X','Y','-X','-Y']:
                I_block,Q_block,laser_block,counter_block = xy_block(tau)
                block_laser.extend(laser_block)
                block_counter.extend(counter_block)
                block_I.extend(I_block)
                block_Q.extend(Q_block)
                meas_time += laser_time+wait_time+tau+pi_pulse*(xy_num*xy_repetition+1)+mw_wait*2

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter_gate'] = block_counter
        sequences['I'] = block_I
        sequences['Q'] = block_Q
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points*4
        }

        return sequences, tagger_config, meas_time

    def get_info(self, measurement):
        info = {
            'parameters': [],
            'channel_map': [],
            'tagger_config': ['click_channel', 'start_channel', 'next_channel', 'sync_channel', 'binwidth', 'n_bins', 'n_histograms']
        }

        if measurement == 'pulsed_ODMR':
            info['parameters'] = ['pi_pulse', 'freq_dev', 'num_freq']
            info['channel_map'] = ['mw_mod', 'mw_switch', 'laser', 'counter_gate', 'sync']
        elif measurement == 'CW_ODMR':
            info['parameters'] = ['pi_pulse', 'freq_dev', 'num_freq']
            info['channel_map'] = ['mw_mod', 'mw_switch', 'laser', 'counter_gate', 'sync']
        elif measurement == 'rabi':
            info['parameters'] = ['laser_time', 'wait_time', 'tau_start', 'tau_end', 'num_points']
            info['channel_map'] = ['laser', 'counter_gate', 'mw', 'sync']
        elif measurement == 'rabi_IQ':
            info['parameters'] = ['laser_time', 'wait_time', 'tau_start', 'tau_end', 'num_points', 'phi']
            info['channel_map'] = ['laser', 'counter_gate', 'I', 'Q', 'sync']
        elif measurement == 'T1':
            info['parameters'] = ['num_points', 'start_time', 'stop_time', 'pi_pulse', 'laser_time', 'wait_time']
            info['channel_map'] = ['laser', 'counter_gate', 'mw', 'sync']
        elif measurement == 'T1_SCC':
            info['parameters'] = ['num_points', 'start_time', 'stop_time', 'pi_pulse',
                                  'G_inti_time', 'G_delay', 'Y_read_time', 'Y_delay',
                                  'R_ionize_time', 'R_delay']
            info['channel_map'] = ['G', 'Y', 'R', 'mw', 'sync', 'counter_gate']
        elif measurement == 'T2':
            info['parameters'] = ['xy_num', 'xy_repetition', 'start_phase', 'laser_time',
                                  'wait_time', 'mw_wait', 'pi_pulse', 'num_points',
                                  'tau_start', 'tau_end']
            info['channel_map'] = ['laser', 'counter_gate', 'I', 'Q', 'sync']
        else:
            print(f"Measurement '{measurement}' info not defined.")

        return info

In [ ]:
class measurement_sequence:
    def __init__(self, parameters=None, channel_map=None, definitions_class=MeasurementDefinitions):
        self.parameters = parameters if parameters is not None else {}
        self.channel_map = channel_map if channel_map is not None else {}
        self.definitions = definitions_class()
        self.sequences = {}
        self.tagger_config = {}

    def set_parameters(self, parameters):
        self.parameters.update(parameters)

    def set_sequences(self, sequences):
        self.sequences = sequences

    def set_channel_map(self, channel_map):
        self.channel_map = channel_map

    def set_tagger_config(self, tagger_config):
        self.tagger_config = tagger_config

    def get_parameters(self):
        return self.parameters

    def get_sequences(self):
        return self.sequences

    def get_tagger_config(self):
        return self.tagger_config

    def get_channel_map(self):
        return self.channel_map

    def create_template(self, measurement):
        """
        Returns template dictionaries for parameters, channel_map, and tagger_config
        based on the measurement definition.
        """
        info = self.definitions.get_info(measurement)

        # Create params dict with placeholders
        params = {param: 0 for param in info['parameters']}

        # Create channel_map dict with placeholders
        channel_map = {ch: ('Digital', 0) for ch in info['channel_map']}

        # Create tagger_config dict with placeholders
        tagger_config = {key: 0 for key in info['tagger_config']}

        return params, channel_map, tagger_config

    def generate_sequence(self, measurement):
        self.sequences = {}
        self.tagger_config = {}

        if hasattr(self.definitions, measurement):
             generator = getattr(self.definitions, measurement)
             # Note: Expects generator to return (sequences, tagger_config, meas_time)
             self.sequences, self.tagger_config, meas_time = generator(self.parameters)
             return meas_time
        else:
            raise ValueError(f"Measurement '{measurement}' is not implemented.")

    def make_pulsestreamer_sequence(self):
        # Helper to convert stored data to PulseStreamer sequence using channel_map
        seq = pulsestreamer.pulse_streamer.createSequence()
        awg_data_map = {}

        for name, pattern in self.sequences.items():
            if name in self.channel_map:
                type, ch = self.channel_map[name]
                if type == 'Analog':
                    seq.setAnalog(ch, pattern)
                elif type == 'Digital':
                    seq.setDigital(ch, pattern)
                elif type == 'AWG':
                    # Check if pattern is a single block (list of tuples) or list of blocks (list of lists)
                    # We assume a block is a list of tuples [(time, val), ...].
                    # If pattern[0] is a list, we assume it's a list of blocks.
                    if not pattern:
                        continue

                    is_multi_step = isinstance(pattern[0], list)

                    if is_multi_step:
                        # List of blocks, implying multiple sequence steps
                        wfs = [AWG_waveform(b) for b in pattern]
                        awg_data_map[ch] = wfs
                    else:
                        # Single block
                        wf = AWG_waveform(pattern)
                        awg_data_map[ch] = [wf] # Store as list for consistent processing
            else:
                print(f"Warning: Sequence '{name}' is not mapped to any channel.")

        if awg_data_map:
            sorted_channels = sorted(awg_data_map.keys())

            # Determine num_waveforms from the first channel
            num_waveforms = len(awg_data_map[sorted_channels[0]])

            waveforms = []
            m1s = []
            m2s = []

            for ch in sorted_channels:
                ch_wfs = awg_data_map[ch]
                if len(ch_wfs) != num_waveforms:
                    raise ValueError(f"Channel {ch} has {len(ch_wfs)} steps, expected {num_waveforms}.")

                if num_waveforms == 1:
                    # Single step: append the array directly (list of arrays)
                    wf = ch_wfs[0]
                    waveforms.append(wf)
                    m1s.append(np.zeros(len(wf)))
                    m2s.append(np.zeros(len(wf)))
                else:
                    # Multi step: append list of arrays (list of lists of arrays)
                    waveforms.append(ch_wfs)
                    m1s.append([np.zeros(len(w)) for w in ch_wfs])
                    m2s.append([np.zeros(len(w)) for w in ch_wfs])

            nreps = [1] * num_waveforms
            trig_waits = [1] * num_waveforms
            jump_tos = [0] * num_waveforms
            goto_states = [0] * num_waveforms
            goto_states[-1] = 1

            AWG_to_send = {
                'waveforms': waveforms,
                'm1s': m1s,
                'm2s': m2s,
                'nreps': nreps,
                'trig_waits': trig_waits,
                'goto_states': goto_states,
                'jump_tos': jump_tos,
                'channels': sorted_channels
            }
            return seq, AWG_to_send
        else:
            return seq

    def make_time_tagger(self):
        tagger = TimeTagger.createTimeTagger()
        counter = TimeTagger.TimeDifferences(tagger=tagger, **self.tagger_config)
        return tagger, counter

In [ ]:
def run_measurement(measurement_name, parameters, channel_map, n_runs=1e6, analyze_window_start=0,analyze_window_length=400,num_curves=1):
    """
    Runs a measurement dynamically based on the name.
    1. Generates sequence using MeasurementDefinitions (via getattr).
    2. Executes hardware sequence.
    3. Fits data using fitting_methods (via getattr).
    """

    # --- 1. Dynamic Sequence Generation ---
    # Initialize your sequence class
    ms = measurement_sequence(parameters, channel_map)

    try:
        meas_time = ms.generate_sequence(measurement_name)
        seq = ms.make_pulsestreamer_sequence()
        tagger, counter = ms.make_time_tagger()
        print(f"Sequence generated. Meas time per shot: {meas_time} ns")
    except ValueError as e:
        print(e)
        return None

    if measurement_name in ['rabi', 'rabi_IQ']:
        x_data = np.linspace(params['tau_start'],params['tau_end'],params['num_points'])
    elif measurement_name in ['T1', 'T1_SCC']:
        x_data = np.geomspace(params['start_time'],params['stop_time'],params['num_points'])
    elif measurement_name == 'T2':
        x_data = np.linspace(params['tau_start'],params['tau_end'],params['num_points'])
    elif measurement_name in ['CW_ODMR', 'pulsed_ODMR']:
        x_data = np.linspace(-params['freq_dev'],params['freq_dev'],params['num_freq'])
    else:
        print(f"No fitting method defined for '{measurement_name}'. Returning raw counts.")
        return counts

    # --- 2. Hardware Execution ---
    # (Assuming global hardware objects: pulsestreamer, mw_source, etc. are set up)
    pulsestreamer.pulse_streamer.stream(seq, int(n_runs))
    pulsestreamer.pulse_streamer.startNow()

    while not pulsestreamer.pulse_streamer.hasFinished():
       time.sleep(0.1)

    counts = get_data(analyze_window_start, analyze_window_length)

    update.clear_plot(measurement_name, num_curves)
    update.update_plot(analyze_window_start, analyze_window_length, x_data, measurement_name,num_curves)

    # --- 3. Dynamic Fitting ---
    if hasattr(fitting_methods, measurement_name):
        fit_func = getattr(fitting_methods, measurement_name)
        print(f"Found fitter: {fit_func.__name__}")

        # Execute the fit
        result = fit_func(x_data, counts)
        print(f"Fit Result: {result}")
        return result
    else:
        print(f"No fitting method defined for '{measurement_name}'. Returning raw counts.")
        return counts


In [ ]:
class update:
    @staticmethod
    def _get_curve_names(name, num_curves):
        # name = 'Rabi', 'T1', 'ODMR', 'b_field_fw'
        if name == 'T1' or name == 'T2' or name == 'T1_SCC':
            countainer_name = 'T1'
        elif name == 'rabi' or name == 'rabi_IQ':
            countainer_name == 'Rabi'
        elif name == 'pulsed_ODMR' or name == 'CW_ODMR':
            countainer_name == 'ODMR'
        names = []
        for i in range(num_curves):
            if i == 0:
                suffix = ""
            elif i == 1:
                suffix = "_2nd"
            elif i == 2:
                suffix = "_3rd"
            else:
                suffix = f"_{i+1}th"
            names.append(f"{countainer_name}{suffix}")
        return names

    @staticmethod
    def update_plot(analyze_window_start, analyze_window_length, x_data, name, num_curves=4):
        counts = get_data(analyze_window_start, analyze_window_length)
        curve_names = update._get_curve_names(name, num_curves)

        if name == 'T2' and num_curves == 4:
            counts_array = np.array(counts)
            # Ensure counts_array has enough elements for 4 curves
            if len(counts_array) >= num_curves:
                mean = (counts_array[0::4] + counts_array[1::4] + counts_array[2::4] + counts_array[3::4]) / 4
                for i, curve_name in enumerate(curve_names):
                    if curve_name in NVscan_gui._NV_spectrum_container:
                        NVscan_gui._NV_spectrum_container[curve_name].setData(x_data/1e9, counts_array[i::num_curves] - mean)
            else:
                print(f"Warning: Not enough data points ({len(counts_array)}) for {num_curves} curves in T1/T2 normalization. Plotting raw data.")
                for i, curve_name in enumerate(curve_names):
                    if curve_name in NVscan_gui._NV_spectrum_container:
                        NVscan_gui._NV_spectrum_container[curve_name].setData(x_data/1e9, counts_array[i::num_curves])
        else:
            for i, curve_name in enumerate(curve_names):
                if curve_name in NVscan_gui._NV_spectrum_container:
                    NVscan_gui._NV_spectrum_container[curve_name].setData(x_data/1e9, counts[i::num_curves])
        return
        # Example:
        # update.update_plot(analyze_window_start, analyze_window_length, mw_length, 'Rabi',num_curves=4)

    @staticmethod
    def clear_plot(name, num_curves=4):
        curve_names = update._get_curve_names(name, num_curves)
        for curve_name in curve_names:
            if curve_name in NVscan_gui._NV_spectrum_container:
                NVscan_gui._NV_spectrum_container[curve_name].clear()
        return


    @staticmethod
    def update_map(name, image_data, **kwargs):
        # Assumes qafm_gui is available in the global scope
        if name in qafm_gui._image_container:
            qafm_gui._image_container[name].setImage(image=image_data, **kwargs)
        return

        # Example:
        # update.update_map('b_field_fw', B_ext, levels=(-2, 2))

In [ ]:
class DataSaver:
    def __init__(self, base_path):
        self.base_path = base_path

    def save(self, data_dict, parameter_dict=None):
        # Create daily folder
        today_str = str(date.today())
        save_dir = os.path.join(self.base_path, today_str)

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        timestr = time.strftime("%Y%m%d-%H%M%S")

        # Save data arrays
        for name, data in data_dict.items():
            filename = f"{name}_{timestr}.txt"
            file_path = os.path.join(save_dir, filename)
            np.savetxt(file_path, data)

        # Save parameters
        if parameter_dict is not None:
            param_filename = f"parameter_{timestr}.txt"
            param_path = os.path.join(save_dir, param_filename)
            with open(param_path, "w") as f:
                f.write(str(parameter_dict))

# Example usage:
# saving_path = 'C:/Users/Anvil/Measurement Data/QUDI/Scanning_data/F-V10-27/t-MoTe2/device_1/'
# saver = DataSaver(saving_path)
#
# data_to_save = {
#     'B': B_ext,
#     'PL': PL,
#     'Top_gate_list': Top_gate_list,
#     'Bot_gate_list': Bot_gate_list
# }
# para_dict = {"delta_v_top":mod_voltage,"pi_pulse":pi_pulse,"delay_time": delay_time}
# saver.save(data_to_save, para_dict)

In [17]:
import numpy as np

In [38]:
scan_params = {
    'angle_deg': 20,
    'scan_length_x': 10,
    'scan_length_y': 20,
    'number_of_points_x': 50,
    'number_of_points_y': 100,
    'start_x': 1,
    'start_y': 2
}
points = generate_2d_scan_points(scan_params, plot=False)
points.shape

(100, 50)

In [ ]:
def run_scanning(scan_points_2d, fast_axis='X', n_runs=100000, mw_power=4, retract_distance=150e-9, central_freq=2.842e9):
    """
    Performs a 2D scan with a selectable fast axis.

    Args:
        scan_points_2d: 2D array of position dicts (from generate_2d_scan_points).
        fast_axis: 'X' for row-by-row (X is fast), 'Y' for column-by-column (Y is fast).
        n_runs: Number of runs for the pulse streamer sequence.
        mw_power: Microwave power in dBm.
        retract_distance: Distance to retract the tip in meters.
        central_freq: Starting central frequency for ODMR.
    """

    # Initialize Data Arrays
    number_of_points_y, number_of_points_x = scan_points_2d.shape
    freq_fit = np.zeros((number_of_points_y, number_of_points_x))
    AFM_height = np.zeros((number_of_points_y, number_of_points_x))
    B_ext = np.zeros((number_of_points_y, number_of_points_x))
    PL = np.zeros((number_of_points_y, number_of_points_x))
    ODMR_spectrum = np.zeros((number_of_points_y, number_of_points_x, num_freq))
    freq_list = np.zeros((number_of_points_y, number_of_points_x, num_freq))

    # Setup Microwave
    mw_source_smb100B.set_power(mw_power)
    mw_source_smb100B.cw_on()
    mw_source_smb100B.set_frequency(central_freq)

    start_time = time.time()
    print(f"Starting Scan. Fast Axis: {fast_axis}, Runs: {n_runs}, MW Power: {mw_power} dBm")

    # Determine Loop Order
    if fast_axis == 'X':
        outer_loop = range(number_of_points_y) # Slow axis: Y
        inner_loop = range(number_of_points_x) # Fast axis: X
    else:
        outer_loop = range(number_of_points_x) # Slow axis: X
        inner_loop = range(number_of_points_y) # Fast axis: Y

    for slow_idx in tqdm(outer_loop, desc=f"Lines ({fast_axis} fast)"):

        # Land Probe (Once per line/column)
        spm.land_probe()
        time.sleep(1.0)

        # --- Pass 1: Get AFM Height ---
        for fast_idx in inner_loop:
            # Map indices based on axis
            if fast_axis == 'X':
                j, i = slow_idx, fast_idx # j=row(y), i=col(x)
            else:
                j, i = fast_idx, slow_idx # j=row(y), i=col(x)

            pos = scan_points_2d[j, i]

            # Move logic
            if fast_idx == 0:
                spm.set_sample_pos_abs(pos, move_time=2)
                time.sleep(2.8)
            else:
                spm.set_sample_pos_abs(pos, move_time=0.2)
                time.sleep(0.07)

            # Record Height
            AFM_height[j, i] = spm._dev.base.getParameter(spm._dev.base.getConst('ID_REG_SET_Z_M'))
            time.sleep(0.05)

        # --- Pass 2: Perform Measurement ---
        for fast_idx in inner_loop:
            if fast_axis == 'X':
                j, i = slow_idx, fast_idx
            else:
                j, i = fast_idx, slow_idx

            pos = scan_points_2d[j, i]
            counter.clear()

            # Move logic (same as above)
            if fast_idx == 0:
                spm.set_sample_pos_abs(pos, move_time=2)
                time.sleep(2.8)
            else:
                spm.set_sample_pos_abs(pos, move_time=0.2)
                time.sleep(0.07)

            # Retract Tip
            spm._dev.base.setParameter(spm._dev.base.getConst('ID_REG_LOOP_ON'), 0, 0)
            spm._dev.zcontrol.setPositionZ(AFM_height[j,i]/1e12 - retract_distance)
            time.sleep(0.1)

            # Run Measurement
            pulsestreamer.pulse_streamer.stream(seq, n_runs)
            pulsestreamer.pulse_streamer.startNow()
            time.sleep(n_runs * meas_time * 1e-9)

            # Get Data
            counts = get_data(analyze_window_start, analyze_window_length)

            # Process / Fit
            current_freqs = np.linspace((central_freq-freq_dev), (central_freq+freq_dev), num_freq)
            freq_list[j,i,:] = current_freqs

            initial_guess = [np.mean(counts), 3.8e6, 0.08, current_freqs[np.argmin(counts)]]
            try:
                fit_results = Lorentzian_fit(current_freqs, counts, initial_guess)
                freq_fit[j,i] = fit_results[0][3]
            except:
                freq_fit[j,i] = current_freqs[np.argmin(counts)]

            ODMR_spectrum[j,i,:] = counts
            PL[j,i] = np.mean(counts)

            # Update GUI
            NVscan_gui._NV_spectrum_container['ODMR'].setData(current_freqs, counts)

            B_ext[j,i] = (2.88e9 - freq_fit[j,i]) * 1e-9 / 0.0028

            # Update Map
            valid_B = B_ext[np.nonzero(B_ext)]
            if len(valid_B) > 0:
                minval, maxval = np.min(np.abs(valid_B)), np.max(np.abs(valid_B))
                qafm_gui._image_container['b_field_fw'].setImage(image=np.abs(B_ext), levels=(minval, maxval))
            else:
                qafm_gui._image_container['b_field_fw'].setImage(image=np.abs(B_ext))

            # Resonance Tracking Logic
            # Prepare central_freq for the NEXT iteration
            if fast_idx == len(inner_loop) - 1:
                # End of line/column: Reset to the value at the start of this line/column (for flyback)
                if fast_axis == 'X':
                    central_freq = freq_fit[j, 0]
                else:
                    central_freq = freq_fit[0, i]
            else:
                # Follow resonance
                central_freq = freq_fit[j, i]

            mw_source_smb100B.set_frequency(central_freq)

    # Cleanup
    mw_source_smb100B.off()
    spm.retract_probe()
    print("Scan Complete")

    return B_ext, AFM_height, freq_fit, PL, ODMR_spectrum

In [ ]:
def run_scanning(scan_points_2d, fast_axis='X', n_runs=100000, mw_power=4, central_freq=2.842e9, retract_distance=None ):

    number_of_points_y, number_of_points_x = scan_points_2d.shape
    freq_fit = np.zeros((number_of_points_y, number_of_points_x))
    AFM_height = np.zeros((number_of_points_y, number_of_points_x))
    B_ext = np.zeros((number_of_points_y, number_of_points_x))
    PL = np.zeros((number_of_points_y, number_of_points_x))
    ODMR_spectrum = np.zeros((number_of_points_y, number_of_points_x, num_freq))
    freq_list = np.zeros((number_of_points_y, number_of_points_x, num_freq))


    start_time = time.time()
    print(f"Starting Scan. Fast Axis: {fast_axis}, Runs: {n_runs}, MW Power: {mw_power} dBm")

    # Determine Loop Order
    if fast_axis == 'X':
        outer_loop = range(number_of_points_y) # Slow axis: Y
        inner_loop = range(number_of_points_x) # Fast axis: X
    else:
        outer_loop = range(number_of_points_x) # Slow axis: X
        inner_loop = range(number_of_points_y) # Fast axis: Y

    for slow_idx in tqdm(outer_loop, desc=f"Lines ({fast_axis} fast)"):

        # Land Probe (Once per line/column)
        spm.land_probe()
        time.sleep(1.0)

        # --- Pass 1: Get AFM Height ---
        if retract_distance:
            for fast_idx in inner_loop:
                # Map indices based on axis
                if fast_axis == 'X':
                    j, i = slow_idx, fast_idx # j=row(y), i=col(x)
                else:
                    j, i = fast_idx, slow_idx # j=row(y), i=col(x)

                pos = scan_points_2d[j, i]

                # Move logic
                if fast_idx == 0:
                    spm.set_sample_pos_abs(pos, move_time=2)
                    time.sleep(2.8)
                else:
                    spm.set_sample_pos_abs(pos, move_time=0.2)
                    time.sleep(0.07)

                # Record Height
                AFM_height[j, i] = spm._dev.base.getParameter(spm._dev.base.getConst('ID_REG_SET_Z_M'))
                time.sleep(0.05)

        # --- Pass 2: Perform Measurement ---
        for fast_idx in inner_loop:
            if fast_axis == 'X':
                j, i = slow_idx, fast_idx
            else:
                j, i = fast_idx, slow_idx

            pos = scan_points_2d[j, i]
            counter.clear()

            # Move logic (same as above)
            if fast_idx == 0:
                spm.set_sample_pos_abs(pos, move_time=2)
                time.sleep(2.8)
            else:
                spm.set_sample_pos_abs(pos, move_time=0.2)
                time.sleep(0.07)

            # Retract Tip
            if retract_distance:
                spm._dev.base.setParameter(spm._dev.base.getConst('ID_REG_LOOP_ON'), 0, 0)

                spm._dev.zcontrol.setPositionZ(AFM_height[j,i]/1e12 - retract_distance)

                time.sleep(0.1)
            else:
                AFM_height[j, i] = spm._dev.base.getParameter(spm._dev.base.getConst('ID_REG_SET_Z_M'))

            # Run Measurement
            pulsestreamer.pulse_streamer.stream(seq, n_runs)
            pulsestreamer.pulse_streamer.startNow()
            time.sleep(n_runs * meas_time * 1e-9)

            # Get Data
            counts = get_data(analyze_window_start, analyze_window_length)

            # Process / Fit
            current_freqs = np.linspace((central_freq-freq_dev), (central_freq+freq_dev), num_freq)
            freq_list[j,i,:] = current_freqs

            initial_guess = [np.mean(counts), 3.8e6, 0.08, current_freqs[np.argmin(counts)]]
            try:
                fit_results = Lorentzian_fit(current_freqs, counts, initial_guess)
                freq_fit[j,i] = fit_results[0][3]
            except:
                freq_fit[j,i] = current_freqs[np.argmin(counts)]

            ODMR_spectrum[j,i,:] = counts
            PL[j,i] = np.mean(counts)

            # Update GUI
            NVscan_gui._NV_spectrum_container['ODMR'].setData(current_freqs, counts)

            B_ext[j,i] = (2.88e9 - freq_fit[j,i]) * 1e-9 / 0.0028

            # Update Map
            valid_B = B_ext[np.nonzero(B_ext)]
            if len(valid_B) > 0:
                minval, maxval = np.min(np.abs(valid_B)), np.max(np.abs(valid_B))
                qafm_gui._image_container['b_field_fw'].setImage(image=np.abs(B_ext), levels=(minval, maxval))
            else:
                qafm_gui._image_container['b_field_fw'].setImage(image=np.abs(B_ext))

            # Resonance Tracking Logic
            # Prepare central_freq for the NEXT iteration
            if fast_idx == len(inner_loop) - 1:
                # End of line/column: Reset to the value at the start of this line/column (for flyback)
                if fast_axis == 'X':
                    central_freq = freq_fit[j, 0]
                else:
                    central_freq = freq_fit[0, i]
            else:
                # Follow resonance
                central_freq = freq_fit[j, i]

            mw_source_smb100B.set_frequency(central_freq)

    # Cleanup
    mw_source_smb100B.off()
    spm.retract_probe()
    print("Scan Complete")

    return B_ext, AFM_height, freq_fit, PL, ODMR_spectrum

### Example Usage

In [ ]:
# Example: Running an X-fast scan
# B_ext, AFM_height, freq_fit, PL, ODMR_spectrum = run_scanning(scan_points_2d, fast_axis='X', n_runs=100000, mw_power=4, retract_distance=150e-9)

# Example: Running a Y-fast scan
# B_ext, AFM_height, freq_fit, PL, ODMR_spectrum = run_scanning(scan_points_2d, fast_axis='Y', n_runs=50000, mw_power=-0, retract_distance=50e-9)

In [35]:
def AWG_waveform(block): # convert [(time, voltage)] sequence to AWG sequence
    AWG_sampling_rate = 1.2e9 # AWG5014C has sampling rate 1.2GS/s

    sampled_wave=[]

    for elements in block:
        num_sample_points = round(elements[0]*1e-9*AWG_sampling_rate)
        sampled_list=[elements[1]] * num_sample_points
        sampled_wave.extend(sampled_list)

    sampled_wave = np.array(sampled_wave)

    return sampled_wave

def send_to_AWG(AWG_to_send):
    waveforms_to_send = AWG_to_send['waveforms']
    m1s_to_send = AWG_to_send['m1s']
    m2s_to_send = AWG_to_send['m2s']
    nreps = AWG_to_send['nreps']
    trig_waits = AWG_to_send['trig_waits']
    goto_states = AWG_to_send['goto_states']
    jump_tos = AWG_to_send['jump_tos']
    channels = AWG_to_send['channels']

    AWG5014C.awg1.make_send_and_load_awg_file(waveforms=waveforms_to_send, m1s=m1s_to_send,  m2s=m2s_to_send, nreps=nreps, trig_waits=trig_waits, goto_states=goto_states, jump_tos=jump_tos, channels=channels)

    return

def generate_2d_scan_points(parameters, plot=False):
    # 1. Define Device/Scan Parameters
    angle_deg = parameters.get('angle_deg', 0)  # Set your rotation angle here, default 0
    theta = np.radians(angle_deg)

    scan_length_x = parameters['scan_length_x']
    scan_length_y = parameters['scan_length_y']
    number_of_points_x = parameters['number_of_points_x']
    number_of_points_y = parameters['number_of_points_y']

    start_x = parameters['start_x']
    start_y = parameters['start_y']

    # Following your style for end points (unrotated basis)
    end_x_rel = scan_length_x - (scan_length_x / number_of_points_x)
    end_y_rel = scan_length_y - (scan_length_y / number_of_points_y)

    # 2. Generate the local relative coordinate vectors
    x_points_rel = np.linspace(0, end_x_rel, number_of_points_x)
    y_points_rel = np.linspace(0, end_y_rel, number_of_points_y)

    scan_points = []

    # 3. Nested loops with Rotation Matrix applied
    # x_rot = x*cos - y*sin
    # y_rot = x*sin + y*cos
    for i in range(number_of_points_y):
        for j in range(number_of_points_x):
            # Current unrotated relative point
            xr = x_points_rel[j]
            yr = y_points_rel[i]

            # Apply rotation and shift by start positions
            rot_x = start_x + (xr * np.cos(theta) - yr * np.sin(theta))
            rot_y = start_y + (xr * np.sin(theta) + yr * np.cos(theta))

            new_position = {'X' : rot_x, 'Y': rot_y}
            scan_points.append(new_position)

    # 4. Reshape into your 2D format
    scan_points_2d = np.reshape(scan_points, (number_of_points_y, number_of_points_x))

    if plot:
        check_x = [[p['X'] for p in row] for row in scan_points_2d]
        check_y = [[p['Y'] for p in row] for row in scan_points_2d]
        import matplotlib.pyplot as plt
        plt.figure()
        plt.scatter(check_x, check_y, s=.1)
        plt.axis('equal')
        plt.show()

    return scan_points_2d

# Example Usage:
# scan_params = {
#     'angle_deg': 20,
#     'scan_length_x': 2000e-9,
#     'scan_length_y': 4000e-9,
#     'number_of_points_x': 50,
#     'number_of_points_y': 100,
#     'start_x': 4.78e-6,
#     'start_y': 6.5e-6
# }
# points = generate_2d_scan_points(scan_params, plot=True)

def fit_Rabi(taus,counts):
    try:
        pulsedmeasurementlogic.fc.set_current_fit('rabi')
        x_fit, y_fit, result = pulsedmeasurementlogic.fc.do_fit(taus/1e9,counts)
        period = 1/result.best_values['frequency']*1e9
        rabi_pi_pulse = period/2
    except:
        rabi_pi_pulse =  mw_time[np.argmin(counts)]
    return rabi_pi_pulse,x_fit,y_fit

def run_rabi_measurement(mw_freq, mw_power, n_runs, rabi_params, analyze_window_start=0, analyze_window_length=200):
    global counter

    # 1. Define Channel Map (consistent with previous cells)
    channel_map = {
        'laser':('Digital',0),
        'counter':('Digital',5),
        'sync':('Digital',6),
        'I':('Analog',0),
        'Q':('Analog',1)
    }

    # 2. Setup Measurement Sequence
    ms = measurement_sequence(rabi_params, channel_map)
    meas_time = ms.generate_sequence('Rabi')
    seq = ms.make_pulsestreamer_sequence()
    tagger, counter = ms.make_time_tagger()

    # 3. Configure Microwave Source
    # Assuming mw_source_sgs100A is the global object available
    mw_source_sgs100A.set_frequency(mw_freq)
    mw_source_sgs100A.set_power(mw_power)
    mw_source_sgs100A.cw_on()

    # Allow a small delay for equipment to settle
    time.sleep(0.05)

    # 4. Start Measurement
    counter.clear()
    # Using startNow() immediately after streaming
    pulsestreamer.pulse_streamer.stream(seq, n_runs)
    pulsestreamer.pulse_streamer.startNow()

    # 5. Wait loop (Blocking)
    while not pulsestreamer.pulse_streamer.hasFinished():
        time.sleep(0.1)

    # 6. Stop Microwave
    mw_source_sgs100A.off()

    # 7. Retrieve and Process Data
    counts = get_data(analyze_window_start, analyze_window_length)

    # Generate x-axis data (tau)
    taus = np.linspace(rabi_params['tau_start'], rabi_params['tau_end'], rabi_params['num_points'])

    return taus, counts




In [ ]:
counts = get_data(analyze_window_start, analyze_window_length)
pi_pulse,x_fit,y_fit=fit_Rabi(taus,counts)
NVscan_gui._NV_spectrum_container['Rabi_2nd'].setData(x_fit, y_fit)
pi_pulse

In [ ]:
# rabi
params = {
    'laser_time':1500,
    'wait_time':500,
    'tau_start':10,
    'tau_end':200,
    'num_points':20,
    'phi':0
    }

taus = np.linspace(params['tau_start'],params['tau_end'],params['num_points'])

channel_map = {
    'laser':('Digital',0),
    'counter_gate':('Digital',5),
    'sync':('Digital',6),
    'mw':('Digital',2),
#     'Q':('Analog',1)
}

ms = measurement_sequence()
ms.set_channel_map(channel_map)
ms.set_parameters(params)
meas_time = ms.generate_sequence('rabi')
# ms.get_sequences()
def handle_timeout():
    if pulsestreamer.pulse_streamer.hasFinished():
         timer.stop()
         mw_source_smb100B.off()

    update.update_plot(analyze_window_start, analyze_window_length, taus, 'Rabi',num_curves=1)

# ms = measurement_sequence(params, channel_map)
# meas_time = ms.generate_sequence('Rabi')
seq = ms.make_pulsestreamer_sequence()
tagger, counter = ms.make_time_tagger()

analyze_window_start = 0
analyze_window_length = 200
integration_time = 100 #sec
n_runs = int(integration_time*1e9/meas_time)

timer = QtCore.QTimer(interval=500, timeout=handle_timeout)
timer.start()


mw_freq = 2.768e9
mw_source_smb100B.set_frequency(mw_freq)
mw_source_smb100B.cw_on()
mw_source_smb100B.set_power(-5)

counter.clear()
time.sleep(0.1)
pulsestreamer.pulse_streamer.stream(seq,n_runs)
pulsestreamer.pulse_streamer.startNow()


In [ ]:
pulsestreamer.pulse_streamer.forceFinal()
timer.stop()
mw_source_smb100B.off()

In [ ]:
# T1
params = {
    'laser_time':1500,
    'wait_time':500,
    'start_time':1e3,
    'stop_time':2e6,
    'num_points':20,
    'pi_pulse':74,
    }
delay_time = np.geomspace(params['start_time'], params['stop_time'], params['num_points'])
channel_map = {
    'laser':('Digital',0),
    'counter_gate':('Digital',5),
    'sync':('Digital',6),
    'mw':('Digital',2),
}

ms = measurement_sequence()
ms.set_channel_map(channel_map)
ms.set_parameters(params)
meas_time = ms.generate_sequence('T1')
# ms.get_sequences()
def handle_timeout():
    if pulsestreamer.pulse_streamer.hasFinished():
         timer.stop()
         mw_source_smb100B.off()

    update.update_plot(analyze_window_start, analyze_window_length, delay_time, 'T1',num_curves=2)

# ms = measurement_sequence(params, channel_map)
# meas_time = ms.generate_sequence('Rabi')
seq = ms.make_pulsestreamer_sequence()
tagger, counter = ms.make_time_tagger()

analyze_window_start = 0
analyze_window_length = 200
integration_time = 100 #sec
n_runs = int(integration_time*1e9/meas_time)

timer = QtCore.QTimer(interval=500, timeout=handle_timeout)
timer.start()


mw_freq = 2.768e9
mw_source_smb100B.set_frequency(mw_freq)
mw_source_smb100B.cw_on()
mw_source_smb100B.set_power(-5)

counter.clear()
time.sleep(0.1)
pulsestreamer.pulse_streamer.stream(seq,n_runs)
pulsestreamer.pulse_streamer.startNow()

In [ ]:
mw_source_sgs100A.set_IQ_mod()

In [ ]:
# rabi IQ
params = {
    'laser_time':1500,
    'wait_time':500,
    'tau_start':10,
    'tau_end':200,
    'num_points':20,
    'phi':np.pi/2
    }
taus = np.linspace(params['tau_start'],params['tau_end'],params['num_points'])

channel_map = {
    'laser':('Digital',0),
    'counter_gate':('Digital',5),
    'sync':('Digital',6),
    'I':('Analog',0),
    'Q':('Analog',1)
}

ms = measurement_sequence()
ms.set_channel_map(channel_map)
ms.set_parameters(params)
meas_time = ms.generate_sequence('rabi_IQ')
# ms.get_sequences()
def handle_timeout():
    if pulsestreamer.pulse_streamer.hasFinished():
         timer.stop()
         mw_source_sgs100A.off()

    update.update_plot(analyze_window_start, analyze_window_length, taus, 'Rabi',num_curves=1)

# ms = measurement_sequence(params, channel_map)
# meas_time = ms.generate_sequence('Rabi')
seq = ms.make_pulsestreamer_sequence()
tagger, counter = ms.make_time_tagger()

analyze_window_start = 0
analyze_window_length = 400
integration_time = 100 #sec
n_runs = int(integration_time*1e9/meas_time)

timer = QtCore.QTimer(interval=500, timeout=handle_timeout)
timer.start()


mw_freq = 2.768e9
mw_source_sgs100A.set_frequency(mw_freq)
mw_source_sgs100A.cw_on()
mw_source_sgs100A.set_power(-5)

counter.clear()
time.sleep(0.1)
pulsestreamer.pulse_streamer.stream(seq,n_runs)
pulsestreamer.pulse_streamer.startNow()

In [ ]:
# T2
params = {
    'xy_num':4,
    'xy_repetition':1,
    'start_phase':'X',
    'laser_time':1500,
    'wait_time':500,
    'mw_wait':0,
    'pi_pulse':50,
    'num_points':20,
    'tau_start':0,
    'tau_end':3e3
}
taus = np.linspace(params['tau_start'],params['tau_end'],params['num_points'])
channel_map = {
    'laser':('Digital',0),
    'counter_gate':('Digital',5),
    'sync':('Digital',6),
    'I':('Analog',0),
    'Q':('Analog',1),
}
ms = measurement_sequence()
ms.set_channel_map(channel_map)
ms.set_parameters(params)
meas_time = ms.generate_sequence('T2')
def handle_timeout():
    if pulsestreamer.pulse_streamer.hasFinished():
         timer.stop()
         mw_source_sgs100A.off()

    counts = get_data(analyze_window_start, analyze_window_length)
    mean = (counts[0::4]+counts[1::4]+counts[2::4]+counts[3::4])/4
    NVscan_gui._NV_spectrum_container['T1'].setData(taus/1e9, counts[0::4]-mean)
    NVscan_gui._NV_spectrum_container['T1_2nd'].setData(taus/1e9, counts[1::4]-mean)
    NVscan_gui._NV_spectrum_container['T1_3rd'].setData(taus/1e9, counts[2::4]-mean)
    NVscan_gui._NV_spectrum_container['T1_4th'].setData(taus/1e9, counts[3::4]-mean)

seq = ms.make_pulsestreamer_sequence()
tagger, counter = ms.make_time_tagger()


analyze_window_start = 0
analyze_window_length = 400
integration_time = 100 #sec
n_runs = int(integration_time*1e9/meas_time)

timer = QtCore.QTimer(interval=500, timeout=handle_timeout)
timer.start()


mw_freq = 2.768e9
mw_source_sgs100A.set_frequency(mw_freq)
mw_source_sgs100A.cw_on()
mw_source_sgs100A.set_power(-5)

counter.clear()
time.sleep(0.1)
pulsestreamer.pulse_streamer.stream(seq,n_runs)
pulsestreamer.pulse_streamer.startNow()

In [ ]:
pulsestreamer.pulse_streamer.forceFinal()
timer.stop()
mw_source_sgs100A.off()